In [1]:
!wget https://github.com/JetBrains/intellij-sdk-docs/archive/d0a992512fb3f7036fddad1a4ba815458baf7d19.zip
!wget https://github.com/JetBrains/kotlin-web-site/archive/589636c4bb9bfdc8bd87ab9f150f6c22d809e598.zip
!wget https://github.com/microsoft/vscode-docs/archive/a03ef83075081dda33db75352b4527387cec2d27.zip
!wget https://github.com/microsoft/TypeScript-Website/archive/a76e4f2f3beb6dcfba85e1b9378567e7b666983a.zip

--2022-04-23 10:24:04--  https://github.com/JetBrains/intellij-sdk-docs/archive/d0a992512fb3f7036fddad1a4ba815458baf7d19.zip
Resolving github.com (github.com)... 64:ff9b::8c52:7903
Connecting to github.com (github.com)|64:ff9b::8c52:7903|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/JetBrains/intellij-sdk-docs/zip/d0a992512fb3f7036fddad1a4ba815458baf7d19 [following]
--2022-04-23 10:24:05--  https://codeload.github.com/JetBrains/intellij-sdk-docs/zip/d0a992512fb3f7036fddad1a4ba815458baf7d19
Resolving codeload.github.com (codeload.github.com)... 64:ff9b::8c52:7909
Connecting to codeload.github.com (codeload.github.com)|64:ff9b::8c52:7909|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘d0a992512fb3f7036fddad1a4ba815458baf7d19.zip’

d0a992512fb3f7036fd     [       <=>          ]  27.88M  15.0MB/s    in 1.9s    

2022-04-23 10:24:07 (15.0 MB/s) - ‘d0a992512fb3f703

In [2]:
!unzip d0a992512fb3f7036fddad1a4ba815458baf7d19.zip
!unzip 589636c4bb9bfdc8bd87ab9f150f6c22d809e598.zip
!unzip a03ef83075081dda33db75352b4527387cec2d27.zip
!unzip a76e4f2f3beb6dcfba85e1b9378567e7b666983a.zip

Archive:  d0a992512fb3f7036fddad1a4ba815458baf7d19.zip
d0a992512fb3f7036fddad1a4ba815458baf7d19
   creating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/
  inflating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.editorconfig  
   creating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/
  inflating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/dependabot.yml  
   creating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/scripts/
  inflating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/scripts/android_studio_releases.main.kts  
   creating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/workflows/
  inflating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/workflows/code-samples.yml  
  inflating: intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19/.github/workflows/dependabot-combine-prs.yml  
  inflating: intellij-sdk-docs-d0a

In [3]:
import marko
from bs4 import BeautifulSoup
import os
import re
from spacy.lang.en import English
import pandas as pd

nlp = English()
nlp.add_pipe("sentencizer")

def get_md_texts(file_path, max_size=1000000):
    with open(file_path, 'r') as f:
        md = f.read(max_size)
    html = marko.convert(md)
    soup = BeautifulSoup(html, features='html.parser')
    texts = []
    for p in soup.select("p"):
        text = p.get_text().replace("\n", " ").replace("  ", " ")
        text = re.sub("{[\w-]+=[^}]+}", "", text)
        if text:
            texts.append(text)
    return texts

def get_md_sentences(file_path, banned={"<", "|"}):
    sentences = []
    for text in get_md_texts(file_path):
        doc = nlp(text)
        for sent in doc.sents:
            sent_text = sent.text.strip()
            skip = False
            for char in banned:
                if char in sent_text:
                    skip = True
                    break
            if not skip and sent_text:
                sentences.append(sent_text)
    return sentences

In [4]:
repos = [
    "intellij-sdk-docs-d0a992512fb3f7036fddad1a4ba815458baf7d19",
    "kotlin-web-site-589636c4bb9bfdc8bd87ab9f150f6c22d809e598",
    "vscode-docs-a03ef83075081dda33db75352b4527387cec2d27",
    "TypeScript-Website-a76e4f2f3beb6dcfba85e1b9378567e7b666983a",
]

In [5]:
all_sentences = []

for repo_path in repos:
    for root, dirs, files in os.walk(repo_path):
        path = root.split(os.sep)
        for file in files:
            if file.endswith(".md"):
                lines = get_md_sentences(root + os.sep + file)
                all_sentences.extend(lines)
len(all_sentences)

66939

In [7]:
df = pd.DataFrame(all_sentences, columns=["text"])
df.to_csv("docs.tsv", sep="\t", index=None, header=None)